In [71]:
import numpy as np
import cvxpy as cp

n = 5
J = np.array([[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]])
e = [[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]]
id = np.array(e)
def E(i,j):
    return np.outer(np.array(e[i]), np.array(e[j])) #+ np.outer(np.array(e[j]), np.array(e[i]))

print(E(1,3))


[[0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [72]:
#primal
X = cp.Variable((n,n), symmetric=True)
constraints = [X >> 0, cp.trace(X) == 1]
for i in range(n):
    constraints.append(cp.trace(E(i,(i+1)%n) @ X) == 0)
constraints
prob = cp.Problem(cp.Maximize(cp.trace(J @ X)), constraints)
print(prob.solve(),"\n")
print(np.round(X.value,2))

2.2360675147417393 

[[0.2  0.   0.12 0.12 0.  ]
 [0.   0.2  0.   0.12 0.12]
 [0.12 0.   0.2  0.   0.12]
 [0.12 0.12 0.   0.2  0.  ]
 [0.   0.12 0.12 0.   0.2 ]]


In [73]:
P = np.round(X.value,2)
print(np.all(np.linalg.eigvals(P) > 0))
np.trace(J @ P)

True


2.2

In [74]:
#dual
Y = cp.Variable((n,n), symmetric=True)
t = cp.Variable()
constraints = [t*id >> Y + J]
for i in range(n):
    constraints.append(cp.trace(E(i,i) @ Y) == 0)
for i in range(n):
    constraints.append(cp.trace(E(i,(i+1)%5) @ Y) == 0)

prob = cp.Problem(cp.Minimize(t), constraints)
print(prob.solve(),"\n")
print(np.round(Y.value,2))

2.236067810454115 

[[ 0.   -0.   -1.38 -1.38 -0.  ]
 [-0.    0.   -0.   -1.38 -1.38]
 [-1.38 -0.    0.   -0.   -1.38]
 [-1.38 -1.38 -0.    0.   -0.  ]
 [-0.   -1.38 -1.38 -0.    0.  ]]


\

Let's try it out with the above matrix.

In [75]:
t = 2.4
M = t*id - np.round(Y.value,2) - J
print(np.all(np.linalg.eigvals(M) > 0))
M

True


array([[ 1.4 , -1.  ,  0.38,  0.38, -1.  ],
       [-1.  ,  1.4 , -1.  ,  0.38,  0.38],
       [ 0.38, -1.  ,  1.4 , -1.  ,  0.38],
       [ 0.38,  0.38, -1.  ,  1.4 , -1.  ],
       [-1.  ,  0.38,  0.38, -1.  ,  1.4 ]])

In [84]:
#dual as mentioned in problem set
J1 = np.array([[1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0], [0,0,0,0,0,0]])
Z = np.array([[0, -1, 0.38, 0.38, -1, 1],
              [-1, 0, -1, 0.38, 0.38,1],
              [ 0.38, -1, 0, -1, 0.38,1],
              [ 0.38, 0.38, -1, 0, -1, 1],
              [-1, 0.38, 0.38, -1, 0,1],
              [1,1,1,1,1,t]])
print(np.all(np.linalg.eigvals(Z+J1) > 0))
Z+J1

False


array([[1.  , 0.  , 1.38, 1.38, 0.  , 1.  ],
       [0.  , 1.  , 0.  , 1.38, 1.38, 1.  ],
       [1.38, 0.  , 1.  , 0.  , 1.38, 1.  ],
       [1.38, 1.38, 0.  , 1.  , 0.  , 1.  ],
       [0.  , 1.38, 1.38, 0.  , 1.  , 1.  ],
       [1.  , 1.  , 1.  , 1.  , 1.  , 2.4 ]])